# Unsupervised Learning Trading Strategy
- Download/Load SP500 stocks prices data.
- Calculate different features and indicators on each stock.
- Aggregate on monthly level and filter top 150 most liquid stocks.
- Calculate Monthly Returns for different time-horizons.
- Download Fama-French Factors and Calculate Rolling Factor Betas.
- For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
- For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe -   ratio optimization.
- Visualize Portfolio returns and compare to SP500 returns.

# Packages Required

- pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

## Download SP500 stocks prices data

In [1]:
from statsmodels.regression.rolling import RollingOLS # A class that runs rolling (windowed) linear regressions
import pandas_datareader.data as web #Tool to pull financial and economic data (e.g., from FRED, Yahoo, IEX, etc.)
import matplotlib.pyplot as plt
#import statsmodels.api as sm #Full statsmodels library (for regression, statistical tests, time series modeling)
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import ta # To easily compute Moving Averages, RSI, MACD, Bollinger Bands
import warnings
warnings.filterwarnings('ignore')


sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].replace('.', '-')
symbols_list = list(sp500['Symbol'].unique())

end_date = '2025-01-01'
start_date = '2015-01-01'

df = yf.download(tickers = symbols_list, start=start_date, end=end_date, auto_adjust=False)
df

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2025-01-01)')


Price        Adj Close                                                  \
Ticker               A        AAPL        ABBV        ABNB         ABT   
Date                                                                     
2015-01-02   37.273827   24.288580   42.761036         NaN   36.744820   
2015-01-05   36.575397   23.604330   41.956303         NaN   36.752995   
2015-01-06   36.005638   23.606550   41.748631         NaN   36.335640   
2015-01-07   36.483501   23.937571   43.435982         NaN   36.630241   
2015-01-08   37.577084   24.857304   43.890255         NaN   37.383141   
...                ...         ...         ...         ...         ...   
2024-12-24  135.560898  257.578674  176.685577  134.990005  113.634544   
2024-12-26  135.291977  258.396667  175.900314  135.320007  114.139534   
2024-12-27  135.003113  254.974930  174.732224  133.384995  113.862282   
2024-12-30  133.887543  251.593094  172.955551  131.809998  111.693764   
2024-12-31  134.055191  249.817383  174.427933  131.410004  112.000725   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2015-01-02  18.539352   74.623978   72.339996   44.687183  38.306625  ...   
2015-01-05  18.428413   73.363998   71.980003   43.874538  36.981152  ...   
2015-01-06  18.469618   72.834824   70.529999   42.844658  36.252129  ...   
2015-01-07  18.577387   74.363586   71.110001   43.295238  36.797043  ...   
2015-01-08  18.900694   75.497559   72.919998   44.059593  36.259499  ...   
...               ...         ...         ...         ...        ...  ...   
2024-12-24  92.669998  358.322815  447.940002  216.161057  49.560802  ...   
2024-12-26  92.930000  357.133789  450.160004  216.131317  49.541222  ...   
2024-12-27  92.339996  352.922638  446.480011  215.070786  49.511856  ...   
2024-12-30  91.889999  349.266388  445.799988  210.679977  49.012627  ...   
2024-12-31  92.349998  348.572815  444.679993  210.580856  49.453121  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2015-01-02  209022  2426600  2228900  2534900  10220400   605900  2283466   
2015-01-05  343789  2385400  1695100  3107200  18502400  1369900  4418651   
2015-01-06  347338  3405900  1975800  4749600  16670700  1333200  5004401   
2015-01-07  348357  2872700  1472000  2833400  13590700  1038600  4554134   
2015-01-08  343147  3004500  1676600  2516800  15487500   821800  4258268   
...            ...      ...      ...      ...       ...      ...      ...   
2024-12-24  191200  1780100   692800   943900   7807000   379300   533000   
2024-12-26  258700  1736500  1218900  1394900   9652400   575700  1040900   
2024-12-27  310700  2320500  1086700  2015000  11943900   552400  1146300   
2024-12-30  320300  2914700  2180100  2642900  11080800   586800  1144600   
2024-12-31  362700  3125200  1612600  2143800  12387800   641600  1217100   

Price                                 
Ticker          ZBH    ZBRA      ZTS  
Date                                  
2015-01-02   936579  411800  1784200  
2015-01-05  2223873  420300  3112100  
2015-01-06  1835563  527500  3977200  
2015-01-07  1505860  467800  2481800  
2015-01-08  1449004  324400  3121300  
...             ...     ...      ...  
2024-12-24   458600   88700  1023600  
2024-12-26  1277300  140100  2167200  
2024-12-27   743400  287200  1800100  
2024-12-30  1532000  211300  1531400  
2024-12-31   683300  327900  1327400  

[2516 rows x 3018 columns]

In [2]:
df.fillna(0, inplace=True)
df.isnull().sum()

Price      Ticker
Adj Close  A         0
           AAPL      0
           ABBV      0
           ABNB      0
           ABT       0
                    ..
Volume     XYL       0
           YUM       0
           ZBH       0
           ZBRA      0
           ZTS       0
Length: 3018, dtype: int64

In [3]:

df = df.stack()
df.index.names = ['Date', 'Ticker']

In [4]:
df.columns=df.columns.str.lower()

In [5]:
df
df.to_csv('sp500_stock_data.xlsx')

# 2. Calculate features and technical indicators for each stock.
- Garman-Klass Volatility - Gives Volatility of Stocks
- RSI - Gives the momentum of the stock, whether it is being bought more or sold more
- Bollinger Bands - Measures Volatility and extreme, Looks at price deviation from moving average
- ATR - ATR tells you how much a stock typically moves per day. Similar to Garman-Klass but different methods
- MACD - MACD tells you when momentum is shifting — i.e., when trends are beginning, strengthening, or ending.
- Dollar Volume - Dollar Volume = Price × Volume. It tells you how much money is flowing through a stock in a day (or over any time window).

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [6]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2*np.log(2) - 1)*(np.log(df['adj close']) - np.log(df['open']))**2
df['rsi'] = df.groupby('Ticker')['adj close'].transform(lambda x : ta.momentum.RSIIndicator(x, window=14).rsi())

In [7]:
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.273827   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288580   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744820   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi  
Date       Ticker                                            
2015-01-02 A         1529200.0         -0.003572        NaN  
           AAPL    212818400.0         -0.006523        NaN  
           ABBV      5086100.0         -0.069835        NaN  
           ABNB            0.0               NaN        NaN  
           ABT       3216600.0         -0.016584        NaN  
...                        ...               ...        ...  
2024-12-31 XYL        641600.0          0.000024  32.070978  
           YUM       1217100.0          0.000034  46.929340  
           ZBH        683300.0          0.000085  41.626042  
           ZBRA       327900.0          0.000088  43.505840  
           ZTS       1327400.0          0.000052  31.259960  

[1265548 rows x 8 columns]

In [8]:
df['bblow'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_lband())
df['bbmid'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_mavg())
df['bbhigh'] = df.groupby(level=1)['adj close'].transform(lambda x : ta.volatility.BollingerBands(close=np.log1p(x), window=20).bollinger_hband())
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.273827   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288580   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744820   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003572        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929340  4.871477   
           ZBH        683300.0          0.000085  41.626042  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259960  5.068102   

Price                 bbmid    bbhigh  
Date       Ticker                      
2015-01-02 A            NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABNB         NaN       NaN  
           ABT          NaN       NaN  
...                     ...       ...  
2024-12-31 XYL     4.799889  4.873531  
           YUM     4.908393  4.945308  
           ZBH     4.679910  4.706731  
           ZBRA    5.989551  6.036588  
           ZTS     5.140142  5.212182  

[1265548 rows x 11 columns]

In [9]:
import ta.volatility


def computeATR(dataframe) :
    atr = ta.volatility.average_true_range(high=dataframe['high'],
                                           low=dataframe['low'],
                                           close=dataframe['close'],
                                           window=20)
    atr = (atr - atr.mean()) / atr.std()
    return atr
df['atr'] = df.groupby(level=1, group_keys=False).apply(computeATR)
#df['atr'] = df.groupby(level=1)['atr'].transform(lambda x: (x - x.mean()) / x.std())
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.273827   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288580   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744820   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003572        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929340  4.871477   
           ZBH        683300.0          0.000085  41.626042  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259960  5.068102   

Price                 bbmid    bbhigh       atr  
Date       Ticker                                
2015-01-02 A            NaN       NaN -1.783827  
           AAPL         NaN       NaN -1.364950  
           ABBV         NaN       NaN -2.518021  
           ABNB         NaN       NaN -0.719302  
           ABT          NaN       NaN -1.978930  
...                     ...       ...       ...  
2024-12-31 XYL     4.799889  4.873531  0.751302  
           YUM     4.908393  4.945308  0.757992  
           ZBH     4.679910  4.706731 -0.542031  
           ZBRA    5.989551  6.036588  0.300231  
           ZTS     5.140142  5.212182  0.623447  

[1265548 rows x 12 columns]

In [10]:
from ta.trend import MACD

df['macd'] = df.groupby(level=1)['adj close'].transform(lambda x : MACD(close=x, window_fast=20).macd())
df['macd'] = df.groupby(level=1)['macd'].transform(
    lambda x: (x - x.mean()) / x.std()
)
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.273827   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288580   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744820   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003572        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929340  4.871477   
           ZBH        683300.0          0.000085  41.626042  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259960  5.068102   

Price                 bbmid    bbhigh       atr      macd  
Date       Ticker                                          
2015-01-02 A            NaN       NaN -1.783827       NaN  
           AAPL         NaN       NaN -1.364950       NaN  
           ABBV         NaN       NaN -2.518021       NaN  
           ABNB         NaN       NaN -0.719302       NaN  
           ABT          NaN       NaN -1.978930       NaN  
...                     ...       ...       ...       ...  
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419  
           YUM     4.908393  4.945308  0.757992 -0.432491  
           ZBH     4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.140142  5.212182  0.623447 -1.900298  

[1265548 rows x 13 columns]

In [11]:
df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2015-01-02 A        37.273827   40.560001   41.310001   40.369999   41.180000   
           AAPL     24.288580   27.332500   27.860001   26.837500   27.847500   
           ABBV     42.761036   65.889999   66.400002   65.440002   65.440002   
           ABNB      0.000000    0.000000    0.000000    0.000000    0.000000   
           ABT      36.744820   44.900002   45.450001   44.639999   45.250000   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.131538  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   volume  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                       
2015-01-02 A         1529200.0         -0.003572        NaN       NaN   
           AAPL    212818400.0         -0.006523        NaN       NaN   
           ABBV      5086100.0         -0.069835        NaN       NaN   
           ABNB            0.0               NaN        NaN       NaN   
           ABT       3216600.0         -0.016584        NaN       NaN   
...                        ...               ...        ...       ...   
2024-12-31 XYL        641600.0          0.000024  32.070978  4.726247   
           YUM       1217100.0          0.000034  46.929340  4.871477   
           ZBH        683300.0          0.000085  41.626042  4.653089   
           ZBRA       327900.0          0.000088  43.505840  5.942514   
           ZTS       1327400.0          0.000052  31.259960  5.068102   

Price                 bbmid    bbhigh       atr      macd  dollar_volume  
Date       Ticker                                                         
2015-01-02 A            NaN       NaN -1.783827       NaN      56.999136  
           AAPL         NaN       NaN -1.364950       NaN    5169.056721  
           ABBV         NaN       NaN -2.518021       NaN     217.486905  
           ABNB         NaN       NaN -0.719302       NaN       0.000000  
           ABT          NaN       NaN -1.978930       NaN     118.193387  
...                     ...       ...       ...       ...            ...  
2024-12-31 XYL     4.799889  4.873531  0.751302 -1.723419      73.972052  
           YUM     4.908393  4.945308  0.757992 -0.432491     161.724855  
           ZBH     4.679910  4.706731 -0.542031 -0.272231      71.836380  
           ZBRA    5.989551  6.036588  0.300231 -0.159019     126.641538  
           ZTS     5.140142  5.212182  0.623447 -1.900298     214.896249  

[1265548 rows x 14 columns]

## 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.
- To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [12]:
# df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume')

In [13]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'close', 'open', 'close', 'high','low','volume']]
last_cols

['adj close',
 'garman_klass_vol',
 'rsi',
 'bblow',
 'bbmid',
 'bbhigh',
 'atr',
 'macd']

In [14]:
df.unstack()[last_cols].resample('M').last().stack('Ticker')
data = pd.concat([df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume'), df.unstack()[last_cols].resample('M').last().stack('Ticker')
], axis=1).dropna()

In [15]:
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2015-02-28 A          102.870265   38.790146         -0.002871  65.073186   
           AAPL      6712.875232   28.651096         -0.005975  63.065350   
           ABBV       520.677101   39.557816         -0.069846  50.513222   
           ABT        225.293286   38.971382         -0.015008  64.968314   
           ACGL        24.453041   18.751719         -0.000874  46.969949   
...                          ...         ...               ...        ...   
2024-12-31 XYL        160.744519  115.293098          0.000024  32.070978   
           YUM        204.737631  132.877213          0.000034  46.929340   
           ZBH        160.263856  105.131538          0.000085  41.626042   
           ZBRA       121.307398  386.220001          0.000088  43.505840   
           ZTS        404.846222  161.892609          0.000052  31.259960   

                      bblow     bbmid    bbhigh       atr      macd  
Date       Ticker                                                    
2015-02-28 A       3.574850  3.640369  3.705888 -1.107361  0.147813  
           AAPL    3.282025  3.361759  3.441493 -0.931137  0.174612  
           ABBV    3.617033  3.677789  3.738546 -0.740117 -0.541717  
           ABT     3.617671  3.661333  3.704995 -1.048068  0.274728  
           ACGL    2.970974  2.990333  3.009693 -1.091787 -0.260423  
...                     ...       ...       ...       ...       ...  
2024-12-31 XYL     4.726247  4.799889  4.873531  0.751302 -1.723419  
           YUM     4.871477  4.908393  4.945308  0.757992 -0.432491  
           ZBH     4.653089  4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.942514  5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.068102  5.140142  5.212182  0.623447 -1.900298  

[57844 rows x 9 columns]

In [16]:
data['dollar_volume'] = (data['dollar_volume'].unstack().rolling(5*12).mean().stack('Ticker'))
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2015-02-28 A                 NaN   38.790146         -0.002871  65.073186   
           AAPL              NaN   28.651096         -0.005975  63.065350   
           ABBV              NaN   39.557816         -0.069846  50.513222   
           ABT               NaN   38.971382         -0.015008  64.968314   
           ACGL              NaN   18.751719         -0.000874  46.969949   
...                          ...         ...               ...        ...   
2024-12-31 XYL        121.846210  115.293098          0.000024  32.070978   
           YUM        200.032725  132.877213          0.000034  46.929340   
           ZBH        177.549157  105.131538          0.000085  41.626042   
           ZBRA       124.840513  386.220001          0.000088  43.505840   
           ZTS        343.347798  161.892609          0.000052  31.259960   

                      bblow     bbmid    bbhigh       atr      macd  
Date       Ticker                                                    
2015-02-28 A       3.574850  3.640369  3.705888 -1.107361  0.147813  
           AAPL    3.282025  3.361759  3.441493 -0.931137  0.174612  
           ABBV    3.617033  3.677789  3.738546 -0.740117 -0.541717  
           ABT     3.617671  3.661333  3.704995 -1.048068  0.274728  
           ACGL    2.970974  2.990333  3.009693 -1.091787 -0.260423  
...                     ...       ...       ...       ...       ...  
2024-12-31 XYL     4.726247  4.799889  4.873531  0.751302 -1.723419  
           YUM     4.871477  4.908393  4.945308  0.757992 -0.432491  
           ZBH     4.653089  4.679910  4.706731 -0.542031 -0.272231  
           ZBRA    5.942514  5.989551  6.036588  0.300231 -0.159019  
           ZTS     5.068102  5.140142  5.212182  0.623447 -1.900298  

[57844 rows x 9 columns]

In [ ]:
data['vol_rank'] = data.groupby('Date')['dollar_volume'].rank(ascending=True)
data = data[data['vol_rank'] < 150].drop(['dollar_volume', 'vol_rank'], axis=1)

In [24]:
data

adj close  garman_klass_vol        rsi     bblow  \
Date       Ticker                                                      
2020-01-31 ACGL     41.991680         -0.001106  53.808722  3.728025   
           AEE      70.076019         -0.009211  82.470776  4.175733   
           AES      16.509386         -0.014468  48.869750  2.856233   
           AIZ     118.163765         -0.003859  48.962175  4.762671   
           AJG      96.089134         -0.001329  81.445509  4.478193   
...                       ...               ...        ...       ...   
2024-12-31 WEC      92.426880         -0.000057  37.642696  4.521057   
           WRB      57.978214         -0.000024  40.423627  4.045766   
           WY       27.728945          0.000067  35.803539  3.300471   
           XYL     115.293098          0.000024  32.070978  4.726247   
           ZBRA    386.220001          0.000088  43.505840  5.942514   

                      bbmid    bbhigh       atr      macd  
Date       Ticker                                          
2020-01-31 ACGL    3.761606  3.795187 -0.505691  0.416570  
           AEE     4.224239  4.272746 -0.727814  1.286745  
           AES     2.875267  2.894301 -0.746324  0.580633  
           AIZ     4.787048  4.811426 -0.527078 -0.287971  
           AJG     4.516665  4.555137 -0.491204  0.279707  
...                     ...       ...       ...       ...  
2024-12-31 WEC     4.552818  4.584579 -0.017752 -1.070965  
           WRB     4.100709  4.155652  1.349429 -1.154067  
           WY      3.401005  3.501538 -0.235063 -1.799598  
           XYL     4.799889  4.873531  0.751302 -1.723419  
           ZBRA    5.989551  6.036588  0.300231 -0.159019  

[8940 rows x 8 columns]

## 4. Calculate Monthly Returns for different time horizons as features.
- To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.